In [15]:
import numpy as np
import pandas as pd    
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import tree

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
import random
from statistics import mean

In [16]:
salary_table = pd.read_csv("salary_table.csv")
seasons = pd.read_csv("Seasons_Stats.csv")

In [17]:
salary_table = salary_table[['Player','season17_18']]
salary_table.rename(columns={'season17_18':'salary17_18'},inplace = True) #variable rename
salary_table['salary17_18'] = salary_table['salary17_18']/1000000 #transform salary to 'million'

seasons = seasons[seasons['Year']>=2017] 
stats17 = seasons[['Year','Player','Pos','Age','G','PER',
                   'MP','PTS','AST','TRB','TOV','BLK','STL']]

stats17.drop_duplicates(subset=['Player'], keep='first',inplace=True) #drop duplicate data

c = ['MPG','PPG','APG','RPG','TOPG','BPG','SPG']
w = ['MP','PTS','AST','TRB','TOV','BLK','STL'] 

for i,s in zip(c,w):
    stats17[i] = stats17[s] / stats17['G']

stats17.drop(w,axis=1,inplace=True)
#stats17.drop(['G'],axis=1,inplace=True)
stats17.loc[stats17['Pos'] == 'PF-C','Pos'] = 'PF'
stats_salary = pd.merge(stats17, salary_table)

<ipython-input-17-4d4bb291a20c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats17.drop_duplicates(subset=['Player'], keep='first',inplace=True) #drop duplicate data
<ipython-input-17-4d4bb291a20c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats17[i] = stats17[s] / stats17['G']
/Users/nachikethinge/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [18]:
stats_salary.describe()

,Year,Age,G,PER,MPG,PPG,APG,RPG,TOPG,BPG,SPG,salary17_18
count,442.0,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,2017.0,26.187783,57.909502,13.641855,21.269669,9.139642,1.975766,3.766571,1.177434,0.407717,0.676931,7.106258
std,0.0,4.325061,22.490430,5.213622,8.608325,6.020116,1.804867,2.437664,0.789349,0.405280,0.403506,7.575776
min,2017.0,19.000000,2.000000,-2.100000,2.600000,0.444444,0.000000,0.000000,0.000000,0.000000,0.000000,0.017224
25%,2017.0,23.000000,42.250000,10.200000,15.179487,4.927978,0.821267,2.057796,0.633537,0.140351,0.404923,1.518316
50%,2017.0,26.000000,66.000000,13.000000,21.253049,7.522693,1.380617,3.169321,0.987179,0.287302,0.608108,3.519283
75%,2017.0,29.000000,76.000000,16.300000,28.007230,12.222891,2.580000,4.908485,1.575185,0.509602,0.900000,10.812701
max,2017.0,40.000000,82.000000,30.800000,37.756757,31.580247,11.185185,14.129870,5.728395,2.641975,2.026316,34.682550


In [19]:
stats_salary.corr()

,Year,Age,G,PER,MPG,PPG,APG,RPG,TOPG,BPG,SPG,salary17_18
Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,NaN,1.000000,0.184243,0.069500,0.189063,0.106059,0.176520,0.085792,0.094436,-0.048671,0.128891,0.216625
G,NaN,0.184243,1.000000,0.275363,0.624499,0.483087,0.317817,0.432422,0.394864,0.269196,0.450836,0.410111
PER,NaN,0.069500,0.275363,1.000000,0.454292,0.677406,0.438792,0.576617,0.528248,0.440554,0.393259,0.521551
MPG,NaN,0.189063,0.624499,0.454292,1.000000,0.861169,0.626363,0.615728,0.739388,0.357551,0.739163,0.669391
PPG,NaN,0.106059,0.483087,0.677406,0.861169,1.000000,0.654111,0.560800,0.822914,0.327264,0.629004,0.703105
APG,NaN,0.176520,0.317817,0.438792,0.626363,0.654111,1.000000,0.228092,0.834185,0.017080,0.666650,0.485655
RPG,NaN,0.085792,0.432422,0.576617,0.615728,0.560800,0.228092,1.000000,0.514544,0.709931,0.421132,0.566535
TOPG,NaN,0.094436,0.394864,0.528248,0.739388,0.822914,0.834185,0.514544,1.000000,0.285149,0.668374,0.584298
BPG,NaN,-0.048671,0.269196,0.440554,0.357551,0.327264,0.017080,0.709931,0.285149,1.000000,0.222870,0.356752


In [20]:
stats_salary.to_csv("stats_salary.csv")

In [21]:
stats_salary.sort_values(by='salary17_18',ascending = False,inplace = True)
stats_salary[['Player','salary17_18']].head(10)

,Player,salary17_18
95,Stephen Curry,34.682550
204,LeBron James,33.285709
280,Paul Millsap,31.269231
174,Gordon Hayward,29.727900
160,Blake Griffin,29.512900
255,Kyle Lowry,28.703704
82,Mike Conley,28.530608
418,Russell Westbrook,28.530608
164,James Harden,28.299399
105,DeMar DeRozan,27.739975


In [22]:
salary_table['salary17_18'].describe()

count    573.000000
mean       5.858946
std        7.162373
min        0.017224
25%        1.312611
50%        2.386864
75%        7.936509
max       34.682550
Name: salary17_18, dtype: float64

In [23]:
stats_salary = stats_salary.dropna()
Y = stats_salary['salary17_18']
X = stats_salary.drop(['salary17_18','Year', 'Player', 'Pos'],axis=1)

In [24]:
X.columns

Index(['Age', 'G', 'PER', 'MPG', 'PPG', 'APG', 'RPG', 'TOPG', 'BPG', 'SPG'], dtype='object')

In [25]:
X.head()

,Age,G,PER,MPG,PPG,APG,RPG,TOPG,BPG,SPG
95,28.0,79.0,24.6,33.392405,25.303797,6.620253,4.468354,3.025316,0.215190,1.810127
204,32.0,74.0,27.0,37.756757,26.405405,8.729730,8.648649,4.094595,0.594595,1.243243
280,31.0,69.0,17.8,33.956522,18.057971,3.652174,7.724638,2.289855,0.898551,1.304348
174,26.0,73.0,22.2,34.465753,21.931507,3.452055,5.383562,1.917808,0.273973,1.000000
160,27.0,61.0,22.7,34.032787,21.573770,4.918033,8.131148,2.327869,0.377049,0.950820


In [26]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2) 

In [27]:
print(len(x_train))
print(len(y_test))

353
89


In [28]:
clf = RandomForestRegressor(random_state=42)
dtree = clf.fit(x_train, y_train)

yp = clf.predict(x_test)
print(clf.score(x_test, y_test))


0.6828064560271947


In [29]:
print(clf.score(x_train, y_train))

0.9359426515918344


In [30]:
df = {'Age': 20, 'G':mean(X['G']), 'PER': mean(X['PER']), 'MPG': mean(X['MPG']), 'PPG': mean(X['G']), 'APG': mean(X['APG']), 'RPG': mean(X['RPG']), 'TOPG': mean(X['TOPG']), 'BPG': mean(X['BPG']), 'SPG': mean(X['SPG'])}

In [31]:
df2 = pd.DataFrame([df])

In [32]:
df2

,Age,G,PER,MPG,PPG,APG,RPG,TOPG,BPG,SPG
0,20,57.909502,13.641855,21.269669,57.909502,1.975766,3.766571,1.177434,0.407717,0.676931


In [33]:
new_pred = clf.predict(df2)
new_pred

array([3.90824719])

In [141]:
"$" + str(6.55773715 * 1000000)

'$6557737.15'